# 10: 工具使用


## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [6]:
# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程
工具使用（也叫函数调用）看起来复杂，但其实很简单！它就是将工具执行结果插入到对话中，让AI能调用外部功能。

**核心原理：**
- 普通对话：我们直接给AI发送文本
- 工具使用：我们让AI告诉我们要调用什么工具，然后把工具结果返回给AI

**工作流程：**
1. AI 说明要调用的工具名称和参数
2. 我们的程序执行这个工具
3. 把执行结果返回给AI继续对话

## 为什么需要工具使用？

工具使用让AI能力大幅扩展，可以处理复杂的实际任务：

**常见工具类型：**
- 📊 数据库查询：执行SQL语句，获取数据
- 🌐 网络请求：调用API，获取实时信息  
- 📁 文件操作：读写文件，处理文档
- 🧮 计算工具：数学运算，数据分析
- 📨 消息发送：邮件、短信、通知

## 实现工具使用的两个关键要素

**1. 系统提示词**
- 告诉AI有哪些工具可用
- 说明每个工具的用途和参数
- 定义工具调用的格式

**2. 控制逻辑**  
- 解析AI的工具调用请求
- 执行对应的工具函数
- 将结果返回给AI继续对话

### 工具使用路线图

*本课程教授我们当前的工具使用格式。但是，我们将在不久的将来更新和改进工具使用功能，包括：*
* *更简化的函数定义和调用格式*
* *更强健的错误处理和边缘情况覆盖*
* *与我们 API 其余部分的更紧密集成*
* *更好的可靠性和性能，特别是对于更复杂的工具使用任务*

## 实战示例：SQL数据库查询工具

下面我们通过一个具体的SQL查询例子来学习工具使用。

### 1. 系统提示词设计

系统提示词需要包含：
- 工具使用的基本说明
- 具体工具的详细描述  
- 调用格式要求

**重要：** 使用 `<function_calls>` 标签是GPT专门训练的格式，建议严格遵循。

In [7]:
# 构建系统提示词 - 优化版本
system_prompt = """
你是一个智能数据分析助手，必须使用提供的工具来查询数据库。

## 可用工具

### sql_query
功能：执行SQL查询语句，获取数据库数据
参数：
- query (字符串): 要执行的SQL查询语句

## 重要规则

1. 当用户询问数据相关问题时，你必须使用sql_query工具查询
2. 不要直接回答，必须先调用工具获取数据
3. 工具调用后立即停止生成，等待工具结果

## 工具调用格式

当你需要查询数据时，严格按以下格式输出：

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 25</parameter>
</invoke>
</function_calls>

## 示例对话

用户："查询年龄大于30的用户"
助手：我将为您查询年龄大于30的用户信息。

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 30</parameter>
</invoke>
</function_calls>

注意：
- 必须使用工具获取数据，不能凭空回答
- SQL语句要准确，避免语法错误
- 工具调用后不要继续生成其他内容
"""

print("✅ 优化版系统提示词构建完成")


✅ 优化版系统提示词构建完成


In [8]:
# 模拟SQL查询工具
def sql_query(query):
    """
    模拟执行SQL查询
    实际项目中这里会连接真实数据库
    """
    # 模拟数据库响应
    mock_data = {
        "users": [
            {"id": 1, "name": "张三", "age": 28, "city": "北京"},
            {"id": 2, "name": "李四", "age": 32, "city": "上海"}, 
            {"id": 3, "name": "王五", "age": 25, "city": "广州"},
            {"id": 4, "name": "赵六", "age": 30, "city": "深圳"}
        ]
    }
    
    print(f"🔍 执行SQL查询: {query}")
    
    # 简单模拟查询结果
    if "age > 25" in query:
        result = [user for user in mock_data["users"] if user["age"] > 25]
    elif "COUNT" in query.upper():
        result = [{"count": len(mock_data["users"])}]
    else:
        result = mock_data["users"]
    
    return result

# 测试工具函数
test_result = sql_query("SELECT * FROM users WHERE age > 25")
print("📋 查询结果:", test_result)


🔍 执行SQL查询: SELECT * FROM users WHERE age > 25
📋 查询结果: [{'id': 1, 'name': '张三', 'age': 28, 'city': '北京'}, {'id': 2, 'name': '李四', 'age': 32, 'city': '上海'}, {'id': 4, 'name': '赵六', 'age': 30, 'city': '深圳'}]


In [9]:
# 工具调用解析函数
def parse_function_calls(response):
    """
    解析AI响应中的工具调用
    """
    import re
    
    # 查找function_calls标签
    pattern = r'<function_calls>(.*?)</function_calls>'
    matches = re.findall(pattern, response, re.DOTALL)
    
    if not matches:
        return None
        
    # 解析工具调用内容
    call_content = matches[0]
    
    # 提取工具名称
    name_pattern = r'<invoke name="([^"]+)">'
    name_match = re.search(name_pattern, call_content)
    if not name_match:
        return None
        
    tool_name = name_match.group(1)
    
    # 提取参数
    param_pattern = r'<parameter name="([^"]+)">([^<]+)</parameter>'
    param_matches = re.findall(param_pattern, call_content)
    
    parameters = {}
    for param_name, param_value in param_matches:
        parameters[param_name] = param_value.strip()
    
    return {
        "name": tool_name,
        "parameters": parameters
    }

print("✅ 工具调用解析函数定义完成")


✅ 工具调用解析函数定义完成


### 2. 完整的工具使用流程

现在我们来演示完整的工具使用过程：


In [10]:
# 优化版工具使用示例
def run_tool_example():
    """
    演示完整的工具使用流程 - 优化版
    """
    print("=" * 60)
    print("🚀 开始工具使用演示（优化版）")
    print("=" * 60)
    
    # 用户请求
    user_query = "帮我查询一下年龄大于25岁的用户有哪些？"
    print(f"👤 用户提问: {user_query}")
    print()
    
    # 第一轮：AI调用工具
    print("🤖 AI第一轮响应:")
    response_1 = get_completion(user_query, system_prompt)
    print(f"原始响应: {response_1}")
    print()
    
    # 解析工具调用
    function_call = parse_function_calls(response_1)
    
    if function_call:
        print("🔧 ✅ 检测到工具调用:")
        print(f"   📋 工具名: {function_call['name']}")
        print(f"   📝 参数: {function_call['parameters']}")
        print()
        
        # 执行工具
        if function_call['name'] == 'sql_query':
            query = function_call['parameters']['query']
            result = sql_query(query)
            print(f"✅ 工具执行结果: {result}")
            print()
            
            # 构建包含结果的新提示
            follow_up_prompt = f"""
之前的查询：{user_query}

工具调用结果：
查询语句：{query}
查询结果：{result}

请根据查询结果给出详细的分析和回答。
"""
            
            # 第二轮：AI分析结果
            print("🤖 AI最终分析:")
            final_response = get_completion(follow_up_prompt, "")
            print(final_response)
    else:
        print("❌ 未检测到工具调用！")
        print("🔍 这可能是因为：")
        print("   1. 系统提示词不够明确")
        print("   2. AI模型没有按预期格式输出")
        print("   3. 解析函数有问题")
        print()
        
        # 备用方案：直接使用更强的提示
        print("🔄 尝试使用更明确的提示...")
        stronger_prompt = f"""
用户问题：{user_query}

请严格按照以下格式调用工具查询数据库：

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 25</parameter>
</invoke>
</function_calls>

必须使用这个格式，不要添加其他内容。
"""
        
        response_2 = get_completion(stronger_prompt, system_prompt)
        print(f"强化提示响应: {response_2}")
        
        # 再次尝试解析
        function_call_2 = parse_function_calls(response_2)
        if function_call_2:
            print("🎉 第二次尝试成功检测到工具调用！")
            query = function_call_2['parameters']['query']
            result = sql_query(query)
            print(f"✅ 工具执行结果: {result}")
        else:
            print("❌ 第二次尝试仍然失败")
    
    print()
    print("=" * 60)
    print("✅ 工具使用演示完成")
    print("=" * 60)

# 运行优化版示例
run_tool_example()


🚀 开始工具使用演示（优化版）
👤 用户提问: 帮我查询一下年龄大于25岁的用户有哪些？

🤖 AI第一轮响应:
原始响应: 我将为您查询年龄大于25岁的用户信息。

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 25</parameter>
</invoke>
</function_calls>

🔧 ✅ 检测到工具调用:
   📋 工具名: sql_query
   📝 参数: {'query': 'SELECT * FROM users WHERE age > 25'}

🔍 执行SQL查询: SELECT * FROM users WHERE age > 25
✅ 工具执行结果: [{'id': 1, 'name': '张三', 'age': 28, 'city': '北京'}, {'id': 2, 'name': '李四', 'age': 32, 'city': '上海'}, {'id': 4, 'name': '赵六', 'age': 30, 'city': '深圳'}]

🤖 AI最终分析:
根据查询结果，我们可以看到年龄大于25岁的用户共有三位，他们的信息如下：

1. **张三**
   - 年龄：28岁
   - 城市：北京

2. **李四**
   - 年龄：32岁
   - 城市：上海

3. **赵六**
   - 年龄：30岁
   - 城市：深圳

### 分析：

- **年龄分布**：这三位用户的年龄分别是28岁、32岁和30岁，均在28岁到32岁之间，显示出一个相对集中的年龄段。
  
- **城市分布**：这三位用户分别来自北京、上海和深圳，都是中国的一线城市。这可能表明年龄大于25岁的用户更倾向于居住在经济发达的大城市。

- **潜在趋势**：从年龄和城市的分布来看，可能存在这样的趋势：年龄较大的用户更可能在经济发达地区工作和生活。这可能与这些城市提供的就业机会和生活条件有关。

### 结论：

年龄大于25岁的用户在样本中显示出集中在一线城市的趋势，且年龄分布较为集中。这可能反映了在这些城市中，较成熟的年龄段的人群比例较高，可能与城市的经济发展水平和生活质量有关。进一步

### 3. 关键要点总结

**系统提示词要点：**
- 📝 清晰描述工具功能和参数
- 🎯 提供具体调用格式示例  
- ⚠️ 说明使用注意事项

**控制逻辑要点：**
- 🔍 解析AI的工具调用请求
- ⚡ 执行对应的工具函数
- 🔄 返回结果给AI继续对话

**最佳实践：**
- 使用标准 `<function_calls>` 格式
- 添加清晰的错误处理
- 提供完整的使用文档


### 3. 工具使用的关键要点

**系统提示词设计：**
- 📝 清晰描述每个工具的功能和参数
- 🎯 给出具体的调用格式示例
- ⚠️ 说明注意事项和限制

**控制逻辑实现：**
- 🔍 准确解析AI的工具调用请求
- ⚡ 快速执行对应的工具函数  
- 🔄 将结果返回给AI继续对话

**最佳实践：**
- 使用标准的 `<function_calls>` 格式
- 工具函数要有清晰的错误处理
- 系统提示词要包含完整的使用说明
